In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
# from sklearn.grid_search import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.preprocessing import StandardScaler
import xgboost
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from statsmodels.stats.weightstats import _zconfint_generic, _tconfint_generic

In [2]:
path = r'C:\Users\Public\Documents\HRS\28.11'
beginning_of_name_EL = '\Feature matrix EL no noise'
beginning_of_name_EL_OM = '\Feature matrix EL+OM no noise'
end_of_name = 'peaks.csv'
list_of_names = [' 3 ', ' 4 ', ' 5 ']

acc = np.zeros((3,3)) # list of accuracies, shape list_of_names*3
f1 = np.zeros((3,3)) # list of f1_scores, shape list_of_names*3
f1_left_confint = np.zeros((3,3))
f1_right_confint = np.zeros((3,3))

for i in range(len(list_of_names)):
    data = pd.read_csv(path + beginning_of_name_EL + list_of_names[i] + end_of_name, index_col=0)
    data1 = pd.read_csv(path + beginning_of_name_EL_OM + list_of_names[i] + end_of_name, index_col=0)
    data.drop(['Label for OM and EL'], axis=1, inplace=True)
    print(i)
    
    X = data.drop('target', 1) #full of EL data ~ 900 sequencies
    y = data['target']

    X1 = data1.drop('target', 1) #full of OM data ~ 200 sequencies
    y1 = data1['target']
    
    target=data1['target']
    data2 = data1.iloc[:,:int((data1.shape[1]-1)/2)]
    data2 = pd.concat([data2, target], axis=1)
    
    X2 = data2.drop('target', 1) #cutted EL data to the size of OM data ~ 200 sequencies
    y2 = data2['target']

    scaler = StandardScaler()
    
    X = scaler.fit_transform(X)
    X1 = scaler.fit_transform(X1)
    X2 = scaler.fit_transform(X2)
    
    X_resampled, y_resampled = SMOTE().fit_resample(X, y)
    X1_resampled, y1_resampled = SMOTE().fit_resample(X1, y1)
    X2_resampled, y2_resampled = SMOTE().fit_resample(X2, y2)
    
    (X_train, X_test, y_train, y_test) = train_test_split(X_resampled, y_resampled, test_size=0.3, stratify=y_resampled)

    (X_train_1, X_test_1, y_train_1, y_test_1) = train_test_split(X1_resampled, y1_resampled, test_size=0.3, stratify=y1_resampled)

    (X_train_2, X_test_2, y_train_2, y_test_2) = train_test_split(X2_resampled, y2_resampled, test_size=0.3, stratify=y2_resampled)
    
    model = RandomForestClassifier(n_estimators = 1000, n_jobs = -1)
    model.fit(X_train, y_train)

    model1 = RandomForestClassifier(n_estimators = 1000, n_jobs = -1)
    model1.fit(X_train_1, y_train_1)

    model2 = RandomForestClassifier(n_estimators = 1000, n_jobs = -1)
    model2.fit(X_train_2, y_train_2)

    y_pred = model.predict(X_test)
    y_pred1 = model1.predict(X_test_1)
    y_pred2 = model2.predict(X_test_2)
    
    acc[i][0]= (accuracy_score(y_test, y_pred) * 100.0)
    acc[i][1]= (accuracy_score(y_test_1, y_pred1) * 100.0)
    acc[i][2]= (accuracy_score(y_test_2, y_pred2) * 100.0)
    
    f1[i][0]= f1_score(y_test, y_pred)
    f1[i][1]= f1_score(y_test_1, y_pred1)
    f1[i][2]= f1_score(y_test_2, y_pred2)
    
    f1_scores = cross_val_score(model, X_resampled, y_resampled, scoring = 'f1',  cv = 20)
    f1_scores_1 = cross_val_score(model1, X1_resampled, y1_resampled, scoring = 'f1',  cv = 20)
    f1_scores_2 = cross_val_score(model2, X2_resampled, y2_resampled, scoring = 'f1',  cv = 20)
    
    mean = f1_scores.mean()
    mean1 = f1_scores_1.mean()
    mean2 = f1_scores_2.mean()
    
    mean_std = f1_scores.std(ddof=1)/np.sqrt(len(f1_scores))
    mean_std1 = f1_scores_1.std(ddof=1)/np.sqrt(len(f1_scores_1))
    mean_std2 = f1_scores_2.std(ddof=1)/np.sqrt(len(f1_scores_2))
    
    tconf=_tconfint_generic(mean, mean_std,len(f1_scores) - 1,0.05, 'two-sided')
    tconf_1=_tconfint_generic(mean1, mean_std1,len(f1_scores_1) - 1,0.05, 'two-sided')
    tconf_2=_tconfint_generic(mean2, mean_std2,len(f1_scores_2) - 1,0.05, 'two-sided')
    
    f1_left_confint[i][0] = tconf[0]
    f1_left_confint[i][1] = tconf_1[0]
    f1_left_confint[i][2] = tconf_2[0]
    
    f1_right_confint[i][0] = tconf[1]
    f1_right_confint[i][1] = tconf_1[1]
    f1_right_confint[i][2] = tconf_2[1]

0


C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype in

1


C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype in

2


C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype in

In [3]:
f1

array([[0.93361884, 0.87878788, 0.81751825],
       [0.96016771, 0.88059701, 0.89552239],
       [0.98319328, 0.921875  , 0.896     ]])

In [4]:
acc

array([[93.41825902, 87.4015748 , 80.31496063],
       [95.96602972, 87.4015748 , 88.97637795],
       [98.3014862 , 92.12598425, 89.76377953]])

In [5]:
f1_left_confint

array([[0.91470849, 0.80858053, 0.76934238],
       [0.93551329, 0.82602774, 0.7820426 ],
       [0.95752485, 0.79771354, 0.8167237 ]])

In [6]:
f1_right_confint

array([[0.95718395, 0.92878439, 0.89066431],
       [0.97247742, 0.94333539, 0.89832107],
       [0.980758  , 0.93348773, 0.90188851]])